In [1]:
import json
import math
import copy
import os
import time
import numpy as np
from pathlib import Path
import pyproj
import networkx as nx
import osmnx as ox
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as geopd
import shapely
from shapely import ops, LineString
import requests
from itertools import chain
from modules import utils
from networkx import bfs_edges

In [2]:
import modules.network_extractor as net_ex
from modules.networker import Networker
extractor = net_ex.NetworkExtractor()
extractor.DATA_BASE_PATH = "/home/user/Desktop/JP/street-network-indices/data"
networker = Networker()


In [14]:
cities = {
    # "buenos_aires": {"search": "Buenos Aires", "country": "Argentina"},
    # "bogota": {"search": "Bogota", "country": "Colombia"},
    # "lima": {"search": "Lima", "country": "Peru"},
    # "chicago": {"search": "Chicago", "country": "United States"},
    # "ottawa": {"search": "Ottawa", "country": "Canada"},
    # "mexico_city": {"search": "Mexico City", "country": "México"},
    # "panama_city": {"search": "Panama City", "country": "Panama"},
    # "san_salvador": {"search": "San Salvador", "country": "El Salvador"},
    # "havana": {"search": "Havana", "country": "Cuba"},
    # "milan": {"search": "Milan", "country": "Italy"},
    # "madrid": {"search": "Madrid", "country": "Spain"},
    "athens": {"search": "Athens", "country": "Greece"},
    # "osaka": {"search": "Osaka", "country": "Japan"},
    # "shanghai": {"search": "Shanghai", "country": "China"},
    # "hanoi": {"search": "Hanoi", "country": "Vietnam"},
    # "dubai": {"search": "Dubai", "country": "United Arab Emirates"},
    # "riyadh": {"search": "Riyadh", "country": "Saudi Arabia"},
    # "doha": {"search": "Doha", "country": "Qatar"},
    # "wellington": {"search": "Wellington", "country": "New Zealand"},
    # "sydney": {"search": "Sydney", "country": "Australia"},
    # "port_moresby": {"search": "Port Moresby", "country": "Papua New Guinea"},
    # "lagos": {"search": "Lagos", "country": "Nigeria"},
    # "alexandria": {"search": "Alexandria", "country": "Egypt"},
    # "pretoria": {"search": "Pretoria", "country": "South Africa"}
}

In [3]:
folder = "milan"
test = extractor.load_graph(f"{folder}/graph/{folder}_drive")
extractor.save_as_shp(test, f"{folder}/shp/{folder}_drive")

test = extractor.load_graph(f"{folder}/graph/{folder}_bike")
extractor.save_as_shp(test, f"{folder}/shp/{folder}_bike")

test = extractor.load_graph(f"{folder}/graph/{folder}_walk")
extractor.save_as_shp(test, f"{folder}/shp/{folder}_walk")

#input_graph: nx.MultiDiGraph
#input_graph = copy.deepcopy(test)
#KEY = extractor.DEFAULT_KEY

/home/geolab/Desktop/Research/notebooks/modules/network_extractor.py:412: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  edges.to_file(f"{self.DATA_BASE_PATH}/{path}_edges.shp", encoding='utf-8')
/home/geolab/Desktop/Research/notebooks/modules/network_extractor.py:412: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  edges.to_file(f"{self.DATA_BASE_PATH}/{path}_edges.shp", encoding='utf-8')
/home/geolab/Desktop/Research/notebooks/modules/network_extractor.py:412: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  edges.to_file(f"{self.DATA_BASE_PATH}/{path}_edges.shp", encoding='utf-8')


In [18]:
city_line_simplified = None
for city,data in cities.items():
    drive_graph = extractor.load_graph(f"{city}/graph/{city}_drive")
    bike_graph = extractor.load_graph(f"{city}/graph/{city}_bike")
    walk_graph = extractor.load_graph(f"{city}/graph/{city}_walk")

    city_graph = networker.add_layer(drive_graph, "drive")
    city_graph = networker.add_layer(bike_graph, "bike")
    city_graph = networker.add_layer(walk_graph, "walk")

    city_line_simplified = networker.get_graph(
        type="complete",
        dual=False,
        rasterize=False,
        simplification="preserve_topology"
    )

    city_line_simplified


Initial: MultiDiGraph with 71318 nodes and 162342 edges
iteration 1
removing self loops
removing dead ends
finished processing initial candidates
Finished 'I' special cases
removing grid
iteration 2
removing dead ends
finished processing initial candidates
Finished 'I' special cases
removing grid
iteration 3
removing dead ends
finished processing initial candidates
Finished 'I' special cases
removing grid
iteration 4
removing dead ends
finished processing initial candidates
Finished 'I' special cases
removing grid
iteration 5
removing dead ends
finished processing initial candidates
Finished 'I' special cases
removing grid
iteration 6
removing dead ends
finished processing initial candidates
Finished 'I' special cases
removing grid
iteration 7
removing dead ends
finished processing initial candidates
Finished 'I' special cases
removing grid
iteration 8
removing dead ends
finished processing initial candidates
Finished 'I' special cases
removing grid
iteration 9
removing dead ends
finis

TypeError: Graph.has_edge() takes 3 positional arguments but 4 were given

In [ ]:
# named STREETS SIMPLIFICATION
input_g = copy.deepcopy(input_graph)
input_g = input_g.to_undirected()

#extractor.save_as_shp(input_g, "test/test", save_nodes=False)
#ox.plot_graph(input_g)
KEY = extractor.DEFAULT_KEY

# remove single disconnected edges
remove_singles = []
for u,v in input_g.edges():
    if len(input_g[u]) <= 1 and len(input_g[v]) <= 1:
        remove_singles.append([u,v,KEY])
input_g.remove_edges_from(remove_singles)

named_streets: dict[str,list] = {}
# sort every street into a natural street
# streets without name get assigned a number sequentially
none_street_counter = 0
# save to which named street each node belongs to
nodes_to_named_street = {}

for u,v,data in input_g.edges(data=True):
    # The natural streets algorithm converts from primal graph to connectivity graph (line graph).
    if "name" in data:
        #street_name = str(data["name"])
        street_name = data["name"]

        # if the street has no name, assign a number sequentially
        if street_name is None:
            street_name = str(none_street_counter)
            none_street_counter += 1

        # if the street name is a combination of multiple streets (list), convert to string
        if type(street_name) == list: street_name = str(street_name)

        # If the current street has not yet been added to the named streets dictionary, add it
        if street_name not in named_streets: named_streets[street_name] = []

        # Add the current edge to the list of edges belonging to the named street
        named_streets[street_name].append([u,v,data])

print("sorted every edge into a named street")

# iterate the named streets and arrange the to maintain road connectivity
named_streets_ordered: dict[str,list] = {}
for named_st_name, named_st_list in named_streets.items():
    edges = {}
    # Convert to undirected by removing 2-way edges (u,v == v,u)
    for u,v,data in named_st_list:
        edge_temp_id = f'{u},{v}'
        reverse_edge_temp_id = f'{v},{u}'
        # If u,v or v,u is not yet in the edges, add it.
        if edge_temp_id not in edges and reverse_edge_temp_id not in edges:
            edges[edge_temp_id] = [u,v,data]
    
    # to support bifurcations with multilines
    st_order_index = 0
    # the street order is a list of continous lines as an array of arrays
    street_order = []
    # temp_edges serves as a stack to build a continous line
    temp_edges = list(edges.values())

    level_index = 0

    # this avoids infinite looping. If the last invalid goes and loop all the edges,
    #   then it is not connected to the main linestring
    last_invalid = None 
    while len(temp_edges) > 0:
        # remove the edges 1 by 1 and append them to the left or right of the array to create a 
        #   continuous line string
        u,v,data = temp_edges.pop(0)
        if len(street_order) == 0: # new line
            if level_index > 0:
                single_street_name = f"{named_st_name}_{level_index}"
            else:
                single_street_name = named_st_name
            street_order.append(u)
            # A single node can belong to more than one named street
            if u not in nodes_to_named_street: nodes_to_named_street[u] = []
            nodes_to_named_street[u].append(single_street_name)
            street_order.append(v)
            # A single node can belong to more than one named street
            if v not in nodes_to_named_street: nodes_to_named_street[v] = []
            nodes_to_named_street[v].append(single_street_name)
            last_invalid = None
            continue

        if u in street_order and street_order[0] == u:
            # add node v to the begining
            street_order.insert(0, v)
            if v not in nodes_to_named_street: nodes_to_named_street[v] = []
            nodes_to_named_street[v].append(single_street_name)
            last_invalid = None
        
        elif u in street_order and street_order[-1] == u:
            # add node v to the end
            street_order.append(v)
            if v not in nodes_to_named_street: nodes_to_named_street[v] = []
            nodes_to_named_street[v].append(single_street_name)
            last_invalid = None
        
        elif v in street_order and street_order[0] == v:
            # add node u to the begining
            street_order.insert(0, u)
            if u not in nodes_to_named_street: nodes_to_named_street[u] = []
            nodes_to_named_street[u].append(single_street_name)
            last_invalid = None

        elif v in street_order and street_order[-1] == v:
            # add node u to the end
            street_order.append(u)
            if u not in nodes_to_named_street: nodes_to_named_street[u] = []
            nodes_to_named_street[u].append(single_street_name)
            last_invalid = None

        else:
            # If it cannot be added to the line, return it to the queue
            temp_edges.append([u,v,data])
            # if the list did a complete loop and no node was added to the line, add a new line!
            if last_invalid == f'{u},{v}':
                named_streets_ordered[single_street_name] = street_order
                level_index += 1
                street_order = []
                continue
            
            elif last_invalid is None:
                last_invalid = f'{u},{v}'
    
    print(f"finished ordering {named_st_name}")
    named_streets_ordered[single_street_name] = street_order
    
# build the graph with the geometries
named_streets_nodes = []
named_streets_edges = []
for st_name, street_nodes in named_streets_ordered.items():
    # build the geometry and the edges from the named street
    agg_length = 0
    u = street_nodes[0]
    line_geometry = []
    for i in range(1, len(street_nodes)):
        v = street_nodes[i]
        # Extract the geometry from the edge
        if input_g.has_edge(u,v):
            # if the edge u,v exist, get the geometry.
            edge_data = input_g[u][v][KEY]
            edge_geometry = edge_data["geometry"]
            geom_len = edge_data["length"]
        else:
            # if the edge u,v does not exist, get the v,u and reverse the geometry.
            edge_data = input_g[v][u][KEY]
            edge_geometry = edge_data["geometry"].reverse()
            geom_len = edge_data["length"]

        # append it to the geometries of the line
        line_geometry.append(edge_geometry)
        agg_length += geom_len

        # Extract the named streets to which U belongs
        edges_u = nodes_to_named_street[u]
        # iterate the named streets to which U belongs
        for edge_out in edges_u:
            # if the edge goes towards a new street, make an edge
            if edge_out != st_name:
                named_streets_edges.append([st_name, edge_out])

        # Extract the named streets to which V belongs
        edges_v = nodes_to_named_street[v]
        # iterate the named streets to which V belongs
        for edge_out in edges_v:
            # if the edge goes towards a new street, make an edge
            if edge_out != st_name:
                named_streets_edges.append([st_name, edge_out])

        # make vertex V the new U to get the next edge    
        u = v
    
    # merge the line strings to a multiline geometry
    named_street_geometry = ops.linemerge(line_geometry)
    named_street_centroid = named_street_geometry.centroid
    # assign the data to the new node (the named street)
    data = {
        "geometry": named_street_geometry, # LineString geometry
        "street_name": st_name, # name of the street
        "lenght": agg_length, # aggregated length (in meters)
        "x": named_street_centroid.coords.xy[0][0],
        "y": named_street_centroid.coords.xy[1][0],
    }

    # Create a dict with the named street name to the node data
    if st_name not in named_streets_nodes:
        named_streets_nodes.append([st_name, copy.deepcopy(data)])

# Build the named streets graph
named_streets_graph = nx.MultiGraph()
# node_id initial index number - node ids are numbers instead of names
node_id_index = 100000
# To map from name to node_id
node_mapping = {}
# add nodes
for node_id, data in named_streets_nodes:
    if not named_streets_graph.has_node(node_id):
        data["id"] = node_id_index
        node_mapping[node_id] = {"id": node_id_index, "node_name": node_id, "data": data}
        named_streets_graph.add_node(node_id_index, **data)
        node_id_index += 1

# add edges
for u,v in named_streets_edges:
    u_index = node_mapping[u]["id"]
    u_data = node_mapping[u]["data"]

    v_index = node_mapping[v]["id"]
    v_data = node_mapping[v]["data"]

    if not named_streets_graph.has_edge(u_index,v_index,KEY):
        edge_data = {
            "geometry": LineString([
                [u_data["x"], u_data["y"]],
                [v_data["x"], v_data["y"]],
            ])
        }
        named_streets_graph.add_edge(u_index,v_index,KEY, **edge_data)
# set the CRS to save as OSMnx-enabled graph 
named_streets_graph.graph["crs"] = "epsg:4326"

# remove unimportant nodes based on cut_level

In [ ]:
#extractor.save_as_shp(named_streets_graph, "temp/athens_big", line_graph=True)
extractor.save_as_shp(named_streets_graph, "temp/athens_big", line_graph=False, save_nodes=True, save_edges=False)
extractor.save_as_shp(named_streets_graph, "temp/athens_big_primal", line_graph=True)

In [ ]:
ox.add

In [ ]:
def sort_degree(n1):
    return n1[1]

input_lod = None
# ox.add_node_elevations_google()
tester = copy.deepcopy(named_streets_graph)
a = list(tester.degree)
a.sort(key=sort_degree, reverse=True)

if input_lod is None:
    max_lod = int(math.log2(tester.number_of_nodes())) + 1
    print(max_lod)
    for lod in range(1, max_lod):
        nodes_to_keep = 2 ** lod
        node_list = list(map(lambda x: x[0], a[0:nodes_to_keep]))
        subg = tester.subgraph(node_list)
        connected = nx.is_connected(subg)
        print(connected)
        if connected: break
else: 
    nodes_to_keep = 2 ** input_lod
    node_list = list(map(lambda x: x[0], a[0:nodes_to_keep]))
    subg = tester.subgraph(node_list)
    connected = nx.is_connected(subg)

print(connected)
print(subg)

extractor.save_as_shp(subg, "temp/sub_big", line_graph=False, save_nodes=True, save_edges=False)
extractor.save_as_shp(subg, "temp/sub_line", line_graph=True)

In [ ]:
extractor.save_as_shp(named_streets_graph, "temp/athens_big_primal", line_graph=True)

In [3]:
from modules import generalize

generalizer = generalize.Generalize()

folder = "athens"
test = extractor.load_graph(f"{folder}/graph/{folder}_drive")
input_graph: nx.MultiDiGraph
input_graph = copy.deepcopy(test)
KEY = extractor.DEFAULT_KEY

extractor.save_as_shp(input_graph, "temp/athens_complete", save_edges=True, save_nodes=False)

topological_ = generalizer.topology_preservation_generalization(input_graph)
extractor.save_as_shp(topological_, "temp/athens_topo_preserve", save_edges=True, save_nodes=False)

named_st_ = generalizer.named_streets_generalization(input_graph)
extractor.save_as_shp(named_st_, "temp/named_streets", save_edges=True, save_nodes=True, line_graph=True)

/home/geolab/Desktop/Research/notebooks/modules/network_extractor.py:410: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  edges.to_file(f"{self.DATA_BASE_PATH}/{path}_edges.shp", encoding='utf-8')


Initial: MultiDiGraph with 71318 nodes and 162342 edges
iteration 1
removing self loops
removing dead ends
finished processing initial candidates
Finished 'I' special cases
removing grid
iteration 2
removing dead ends
finished processing initial candidates
Finished 'I' special cases
removing grid
iteration 3
removing dead ends
finished processing initial candidates
Finished 'I' special cases
removing grid
iteration 4
removing dead ends
finished processing initial candidates
Finished 'I' special cases
removing grid
iteration 5
removing dead ends
finished processing initial candidates
Finished 'I' special cases
removing grid
iteration 6
removing dead ends
finished processing initial candidates
Finished 'I' special cases
removing grid
iteration 7
removing dead ends
finished processing initial candidates
Finished 'I' special cases
removing grid
iteration 8
removing dead ends
finished processing initial candidates
Finished 'I' special cases
removing grid
iteration 9
removing dead ends
finis

/home/geolab/Desktop/Research/notebooks/modules/network_extractor.py:410: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  edges.to_file(f"{self.DATA_BASE_PATH}/{path}_edges.shp", encoding='utf-8')


sorted every edge into a named street
finished ordering Άρεως
finished ordering Δεξίππου
finished ordering Βρυσακίου
finished ordering Πανός
finished ordering ['Πανός', 'Αδριανού']
finished ordering 0
finished ordering 1
finished ordering 2
finished ordering Ποσειδώνος
finished ordering Εθνάρχου Μακαρίου
finished ordering Ανθυπολοχαγού Γιάνναρου
finished ordering Δήμος Κομνηνού (Σαλαμίνος)
finished ordering Ελευθερίας
finished ordering Νίκης
finished ordering Δαβάκη
finished ordering Κορυτσάς
finished ordering Μαλτέζου
finished ordering Ταξιαρχών
finished ordering Ναυαρίνου
finished ordering Σόλωνος
finished ordering 3
finished ordering Αριστοτέλους
finished ordering Ρήγα Φεραίου
finished ordering Θουκυδίδου
finished ordering 4
finished ordering 5
finished ordering 6
finished ordering Δήμητρος
finished ordering Καραολή & Δημητρίου
finished ordering Αχιλλέως
finished ordering Μουσών
finished ordering 7
finished ordering Λεωφόρος Ποσειδώνος
finished ordering Κανάρη
finished ordering Καμα

/home/geolab/Desktop/Research/notebooks/modules/network_extractor.py:408: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  nodes.to_file(f"{self.DATA_BASE_PATH}/{path}_nodes.shp", encoding='utf-8')


In [5]:
named_st_ = generalizer.named_streets_generalization(input_graph)

def sort_degree(n1):
    return n1[1]

input_lod = 10
# ox.add_node_elevations_google()
tester = copy.deepcopy(named_st_)
a = list(tester.degree)
a.sort(key=sort_degree, reverse=True)

if input_lod is None:
    max_lod = int(math.log2(tester.number_of_nodes())) + 1
    print(max_lod)
    for lod in range(1, max_lod):
        nodes_to_keep = 2 ** lod
        node_list = list(map(lambda x: x[0], a[0:nodes_to_keep]))
        subg = tester.subgraph(node_list)
        connected = nx.is_connected(subg)
        print(connected)
        if connected: break
else: 
    nodes_to_keep = 2 ** input_lod
    node_list = list(map(lambda x: x[0], a[0:nodes_to_keep]))
    subg = tester.subgraph(node_list)
    connected = nx.is_connected(subg)

print(connected)
print(subg)

extractor.save_as_shp(subg, "temp/named_streets_SIMPLIFIED", save_edges=True, save_nodes=True, line_graph=True)

sorted every edge into a named street
finished ordering Άρεως
finished ordering Δεξίππου
finished ordering Βρυσακίου
finished ordering Πανός
finished ordering ['Πανός', 'Αδριανού']
finished ordering 0
finished ordering 1
finished ordering 2
finished ordering Ποσειδώνος
finished ordering Εθνάρχου Μακαρίου
finished ordering Ανθυπολοχαγού Γιάνναρου
finished ordering Δήμος Κομνηνού (Σαλαμίνος)
finished ordering Ελευθερίας
finished ordering Νίκης
finished ordering Δαβάκη
finished ordering Κορυτσάς
finished ordering Μαλτέζου
finished ordering Ταξιαρχών
finished ordering Ναυαρίνου
finished ordering Σόλωνος
finished ordering 3
finished ordering Αριστοτέλους
finished ordering Ρήγα Φεραίου
finished ordering Θουκυδίδου
finished ordering 4
finished ordering 5
finished ordering 6
finished ordering Δήμητρος
finished ordering Καραολή & Δημητρίου
finished ordering Αχιλλέως
finished ordering Μουσών
finished ordering 7
finished ordering Λεωφόρος Ποσειδώνος
finished ordering Κανάρη
finished ordering Καμα

/home/geolab/Desktop/Research/notebooks/modules/network_extractor.py:408: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  nodes.to_file(f"{self.DATA_BASE_PATH}/{path}_nodes.shp", encoding='utf-8')


In [9]:
print(topological_)

MultiDiGraph with 26392 nodes and 45662 edges
